In [ ]:
! pip install transformers==4.38.0
! pip install datasets==2.21.0
! pip install torch
! pip install librosa
! pip install IPython
! pip install matplotlib
! pip install seaborn
!pip install Levenshtein



import torch
import numpy as np
!pip install jiwer
from jiwer import wer
import seaborn as sns
from google.colab import files

from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperFeatureExtractor,WhisperConfig
from datasets import load_dataset, load_metric, Audio
from IPython.display import Audio as AudioDis
import librosa.display
import matplotlib.pyplot as plt
from text_unidecode import unidecode
import numpy as np
import pandas as pd

import threading
import time

def keep_session_alive():
    while True:
        time.sleep(60)  # Espera 60 segundos
        print("Manteniendo la sesión activa...")

# Ejecuta la función en un hilo separado para no bloquear la ejecución del notebook
thread = threading.Thread(target=keep_session_alive)
thread.start()



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 116.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.38.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import pandas as pd


# Configuración inicial
config = WhisperConfig.from_pretrained("openai/whisper-small")
#set dropout
config.activation_dropout = 0.05
dropout = config.activation_dropout
#set the configuration and the processor
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small", config=config)
processor = WhisperProcessor.from_pretrained("openai/whisper-small", config=config)
config.activation_dropout = 0.0
model_nd = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small", config=config)

!pip install datasets
from dataset_loader import Dataset_loader
from model_evaluator import Whisper_Evaluator
#from whisper_trainer import Whisper_Builder

sampling_rate = 16000
data_loader = Dataset_loader()

list_rows_wer = []
score_wer_rows = []

# Valores de num_transcriptions y numero_particion
num_transcriptions_values = [15]
numero_particion_values = range(1, 11)  # Incluye valores de 1 a 10

for num_transcriptions in num_transcriptions_values:
    for numero_particion in numero_particion_values:
        dataset_test = data_loader.load_dataset_raw_uq(partition_type="test", partition=numero_particion)
        #subset_dataset_test = dataset_test.select(range(2))
        audios = dataset_test["audio"]
        texts = dataset_test["sentence"]
        #subset_audios, subset_texts = load_selected_subset_1(audio, texts)

        whisper_evaluator = Whisper_Evaluator(model, processor, sampling_rate=sampling_rate)
        print("número partición:",numero_particion)
        # Obtener las transcripciones usando transcriptions_MCD_dataset
        certainties, transcriptions_list_mc = whisper_evaluator.transcriptions_MCD_dataset(audios, num_transcriptions=num_transcriptions)
        print("transcripciones MC Dropout:", transcriptions_list_mc)

        # No dropout
        whisper_evaluator = Whisper_Evaluator(model_nd, processor, sampling_rate=sampling_rate)
        transcriptions_list, gt_list = whisper_evaluator.transcribe_dataset(dataset_test)
        wers_all = whisper_evaluator.compute_wers(transcriptions_list, gt_list)
        avg_wer = sum(wers_all) / len(wers_all)
        std_wer = torch.tensor(wers_all).std().item()
        # MC dropout
        from uq_mc_dropout import uq_MC_dropout

        transcriptions_all = transcriptions_list_mc
        gt_texts = texts
        uq_mc = uq_MC_dropout(num_transcriptions)
        print("Word Error Rates:", wers_all)

        distancias_totales, medoides, longitudes_maximas, longitudes = uq_mc.uncertainty_MCD(transcriptions_all)
        Score = uq_mc.dividir_distancias(distancias_totales, longitudes_maximas)
        print("Incertidumbre normalizada:", Score)
        ece = uq_mc.compute_ece(Score, wers_all)
        print("Expected Calibration Error (ECE):", ece)

        # Guardar Score y wers_all como cadenas de texto
        score_wer_dict = {
            "partition": numero_particion,
            "num_transcriptions": num_transcriptions,
            "scores": str(Score),
            "wers_all": str(wers_all)
        }
        score_wer_rows.append(score_wer_dict)


        row_dictionary_wer = {
            "partition": numero_particion,
            "num_transcriptions": num_transcriptions,
            "dropout":dropout,
            "wer mean:": avg_wer,
            "wer std": std_wer,
            "correlacion:":ece

        }
        list_rows_wer.append(row_dictionary_wer)

# Generar los DataFrames finales
data_frame_test_wer = pd.DataFrame(list_rows_wer)
data_frame_score_wer = pd.DataFrame(score_wer_rows)

# Guardar los DataFrames en archivos CSV
data_frame_test_wer.to_csv("data_frame_test_wer.csv", index=False)
data_frame_score_wer.to_csv("data_frame_score_wer.csv", index=False)

# Descargar los archivos CSV
files.download("data_frame_test_wer.csv")
files.download("data_frame_score_wer.csv")


print(data_frame_test_wer)
print(data_frame_score_wer)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ModuleNotFoundError: No module named 'dataset_loader'

In [ ]:
# Descargar los archivos CSV
files.download("data_frame_test_wer.csv")
files.download("data_frame_score_wer.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>